In [1]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import time
import random
import warnings
import numpy as np
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from gensim.models import FastText
import os
import json
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
warnings.filterwarnings("ignore")

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [2]:
def load_data():
    words = []
    y_train = []
    with open('20news.json', 'r') as f:
        lines = []
        for l in f:
            lines.append(l)
        random.shuffle(lines)
        for l in lines:
            data = json.loads(l)
            words.append(data["text"])
            onehot = [0 for i in range(20)]
            onehot[data["label"]] = 1
            y_train.append(onehot)
    model = FastText.load('fasttext.model')
    len1 = len(words)
    labels_maxn = 20
    words_maxn = 32
    dict_vec={}
    for word in model.wv.index2word:
        dict_vec[word] = list(model.wv[word])
    x_train=[]
    maxn = 340

    zerolist = [0 for i in range(labels_maxn)]
    for i in range(len1):
        if y_train[i] == zerolist:
            continue
        embedding_one=[]
        words_one=words[i]
        len_word = len(words_one)
        if len_word >= maxn:
            words_one = words_one[0:maxn]
            for j in range(maxn):
                if words_one[j] in dict_vec:
                    embedding_one.append(dict_vec[words_one[j]])
                else:
                    dict_vec[words_one[j]]=[0 for temp in range(words_maxn)]
                    embedding_one.append(dict_vec[words_one[j]])
        else:
            for k in range(maxn-len_word):
                words_one.append("#")
            for j in range(maxn):
                if words_one[j] in dict_vec:
                    embedding_one.append(dict_vec[words_one[j]])
                else:
                    dict_vec[words_one[j]] = [0 for temp in range(words_maxn)]
                    embedding_one.append(dict_vec[words_one[j]])
        x_train.append(embedding_one)

    y_train_suc = []
    for y_train_one in y_train:
        if y_train_one == zerolist:
            continue
        y_train_suc.append(y_train_one)
    y_train = y_train_suc

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    return x_train,  y_train

In [3]:
x_all, y_all=load_data()

print(np.array(x_all).shape,flush=True)
print(np.array(y_all).shape,flush=True)
# 有效数据18828个
maxn = len(x_all)
trainn = int(maxn * 0.8)
x_train = x_all[0:trainn]
x_test = x_all[trainn:maxn]
y_train = y_all[0:trainn]
y_test = y_all[trainn:maxn]
#训练集：验证集；测试集=6:2:2
print(np.array(x_train).shape,flush=True)
print(np.array(y_train).shape,flush=True)
print(np.array(x_test).shape,flush=True)
print(np.array(y_test).shape,flush=True)

(18828, 340, 32)
(18828, 20)
(15062, 340, 32)
(15062, 20)
(3766, 340, 32)
(3766, 20)


In [4]:
model = Sequential()
model.add(Bidirectional(LSTM(input_dim=32, output_dim=64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(output_dim=20))
model.add(Activation("sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(x_train, y_train, batch_size=128, nb_epoch=20, validation_split=0.25)#验证集占整体0.2
print(model.summary(),flush=True)

Train on 11296 samples, validate on 3766 samples
Epoch 1/20
11296/11296 [==============================] - 98s 9ms/step - loss: 0.2499 - val_loss: 0.1973
Epoch 2/20
11296/11296 [==============================] - 100s 9ms/step - loss: 0.2003 - val_loss: 0.1955
Epoch 3/20
11296/11296 [==============================] - 100s 9ms/step - loss: 0.1953 - val_loss: 0.1876
Epoch 4/20
11296/11296 [==============================] - 101s 9ms/step - loss: 0.1889 - val_loss: 0.1803
Epoch 5/20
11296/11296 [==============================] - 97s 9ms/step - loss: 0.1814 - val_loss: 0.1735
Epoch 6/20
11296/11296 [==============================] - 95s 8ms/step - loss: 0.1747 - val_loss: 0.1676
Epoch 7/20
11296/11296 [==============================] - 95s 8ms/step - loss: 0.1668 - val_loss: 0.1583
Epoch 8/20
11296/11296 [==============================] - 102s 9ms/step - loss: 0.1584 - val_loss: 0.1522
Epoch 9/20
11296/11296 [==============================] - 100s 9ms/step - loss: 0.1527 - val_loss: 0.1488
E

In [5]:
#进行测试并得到精度
predicted = model.predict(x_train)
predicted=predicted.tolist()
predicted_last=[]
for i in range(len(predicted)):
    ls = predicted[i]
    mx = 0
    for j in range(len(ls)):
        if ls[j]>ls[mx]:
            mx = j
    predicted_last.append(mx)
    
true_predict = 0
false_predict = 0
for i in range(len(y_train)):
    ls = y_train[i]
    label = predicted_last[i]
    if ls[label] == 1:
        true_predict += 1
    else:
        false_predict += 1

print(true_predict, false_predict)
print(true_predict/(true_predict+false_predict))

10373 4689
0.68868676138627


In [6]:
#进行测试并得到精度
predicted = model.predict(x_test)
predicted=predicted.tolist()
predicted_last=[]
for i in range(len(predicted)):
    ls = predicted[i]
    mx = 0
    for j in range(len(ls)):
        if ls[j]>ls[mx]:
            mx = j
    predicted_last.append(mx)

In [7]:
true_predict = 0
false_predict = 0
for i in range(len(y_test)):
    ls = y_test[i]
    label = predicted_last[i]
    if ls[label] == 1:
        true_predict += 1
    else:
        false_predict += 1

print(true_predict, false_predict)
print(true_predict/(true_predict+false_predict))

2173 1593
0.5770047796070101
